In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import os
import sys
import platform
from copy import deepcopy
from typing import List, Dict, Optional, Union, Callable
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

# # required to enable non-blocking interaction:
# %gui qt5 ## TODO 2024-01-18 - this causes kernel to crash when running notebook remotely via VSCode's ssh remote

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.exception_helpers import CapturedException

# Jupyter interactivity:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget
from pyphocorehelpers.gui.Jupyter.simple_widgets import code_block_widget

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler, SavingOptions
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionTables, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import build_vscode_workspace, build_windows_powershell_run_script

from pyphocorehelpers.exception_helpers import CapturedException
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult, BatchSessionCompletionHandler

from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots, BatchPhoJonathanFiguresHelper
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsHelpers
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget

import inspect
from jinja2 import Template
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import MAIN_get_template_string, write_test_script
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import export_session_h5_file_completion_function, curr_runtime_context_header_template, export_rank_order_results_completion_function, figures_rank_order_results_completion_function, compute_and_export_marginals_dfs_completion_function, determine_session_t_delta_completion_function, perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function, compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function, reload_exported_kdiba_session_position_info_mat_completion_function, compute_and_export_session_wcorr_shuffles_completion_function, compute_and_export_session_instantaneous_spike_rates_completion_function
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import ProcessingScriptPhases

# BATCH_DATE_TO_USE = '2024-06-11_GL' # used for filenames throught the notebook
BATCH_DATE_TO_USE = '2024-06-11_Lab' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = '2024-06-11_Apogee' # used for filenames throught the notebook

# scripts_output_path = Path('/home/halechr/cloud/turbo/Data/Output/gen_scripts/').resolve() # Greatlakes
# scripts_output_path = Path('output/gen_scripts/').resolve() # Apogee
# # scripts_output_path = Path('/home/halechr/FastData/gen_scripts/').resolve() # Lab
# assert scripts_output_path.exists()
known_scripts_output_paths = [Path(v).resolve() for v in ['/home/halechr/cloud/turbo/Data/Output/gen_scripts/', '/home/halechr/FastData/gen_scripts/', 'output/gen_scripts/']]
scripts_output_path = find_first_extant_path(known_scripts_output_paths)
assert scripts_output_path.exists(), f"scripts_output_path: {scripts_output_path} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'scripts_output_path: {scripts_output_path}')

collected_outputs_path = scripts_output_path.joinpath('../collected_outputs').resolve()
collected_outputs_path.mkdir(exist_ok=True)
assert collected_outputs_path.exists()
print(f'collected_outputs_path: {collected_outputs_path}')

Automatic pdb calling has been turned OFF
scripts_output_path: /home/halechr/cloud/turbo/Data/Output/gen_scripts
collected_outputs_path: /home/halechr/cloud/turbo/Data/Output/collected_outputs


## Build Processing Scripts:

In [2]:
# Get the generated code for user-contributed functions:
phase_any_run_custom_user_completion_functions_dict = {
# "export_rank_order_results_completion_function": export_rank_order_results_completion_function, # ran 2024-05-28 6am
# "figures_rank_order_results_completion_function": figures_rank_order_results_completion_function,
# "compute_and_export_marginals_dfs_completion_function": compute_and_export_marginals_dfs_completion_function, # ran 2024-05-28 6am
# "determine_session_t_delta_completion_function": determine_session_t_delta_completion_function,  # ran 2024-05-28 6am
# 'perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function': perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function, # ran 2024-05-28 6am
# 'compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function': compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function, # ran 2024-05-28 6am
# 'reload_exported_kdiba_session_position_info_mat_completion_function': reload_exported_kdiba_session_position_info_mat_completion_function,
# 'export_session_h5_file_completion_function': export_session_h5_file_completion_function, # ran 2024-05-28 3am
    # 'compute_and_export_session_wcorr_shuffles_completion_function': compute_and_export_session_wcorr_shuffles_completion_function,
# 'compute_and_export_session_instantaneous_spike_rates_completion_function': compute_and_export_session_instantaneous_spike_rates_completion_function,
}


In [3]:
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import BatchScriptsCollection, generate_batch_single_session_scripts, display_generated_scripts_ipywidget

# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # Other file
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), 
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # 
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # Other tab
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'), 
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), #
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), #
]

## Setup Functions:
force_recompute_override_computations_includelist = []
# force_recompute_override_computations_includelist = ['directional_decoders_evaluate_epochs',]
# force_recompute_override_computations_includelist = ['split_to_directional_laps','merged_directional_placefields','directional_decoders_evaluate_epochs','directional_decoders_epoch_heuristic_scoring']
# force_recompute_override_computations_includelist = ['directional_decoders_evaluate_epochs','directional_decoders_epoch_heuristic_scoring']
# force_recompute_override_computations_includelist = ['split_to_directional_laps','merged_directional_placefields']
# force_recompute_override_computations_includelist = ['long_short_decoding_analyses', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding',]
# force_recompute_override_computations_includelist = ['wcorr_shuffle_analysis','trial_by_trial_metrics']

extra_extended_computations_include_includelist = [
	# 'wcorr_shuffle_analysis'
]

custom_phase_extended_computations_include_includelist = None
# custom_phase_extended_computations_include_includelist=['lap_direction_determination', 'pf_computation', 'pfdt_computation', 'firing_rate_trends',
# 	# 'pf_dt_sequential_surprise',
# 	'extended_stats',
# 	'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 
# 	# 'ratemap_peaks_prominence2d',
# 	'long_short_inst_spike_rate_groups',
# 	# 'long_short_endcap_analysis',
# 	# 'spike_burst_detection',
# 	'split_to_directional_laps',
# 	'merged_directional_placefields',
# 	'rank_order_shuffle_analysis',
# 	# 'directional_train_test_split',
# 	# 'directional_decoders_decode_continuous',
# 	# 'directional_decoders_evaluate_epochs',
# 	# 'directional_decoders_epoch_heuristic_scoring',
#     # 'wcorr_shuffle_analysis',
#     # 'trial_by_trial_metrics',
# ]

active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'W:/Data'), Path(r'/home/halechr/FastData'), Path(r'/Volumes/MoverNew/data')] # Path(r'/home/halechr/cloud/turbo/Data'), , Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/home/halechr/turbo/Data'), 
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

## Different run configurations:

# active_phase = ProcessingScriptPhases.clean_run
active_phase = ProcessingScriptPhases.continued_run
# active_phase = ProcessingScriptPhases.final_run
# active_phase = ProcessingScriptPhases.figure_run

custom_user_completion_functions_dict = active_phase.get_custom_user_completion_functions_dict(extra_run_functions=phase_any_run_custom_user_completion_functions_dict)
# print(f'custom_user_completion_functions_dict: {custom_user_completion_functions_dict}')
custom_user_completion_function_template_code, custom_user_completion_functions_dict = MAIN_get_template_string(BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, collected_outputs_path=collected_outputs_path, override_custom_user_completion_functions_dict=custom_user_completion_functions_dict)
# print(f'custom_user_completion_function_template_code: {custom_user_completion_function_template_code}')
active_phase_dict = active_phase.get_run_configuration(custom_user_completion_function_template_code=custom_user_completion_function_template_code, extra_extended_computations_include_includelist=extra_extended_computations_include_includelist)

## Completely replace with custom functions:
if custom_phase_extended_computations_include_includelist is not None:
	print(f'WARNING: custom_phase_extended_computations_include_includelist: {custom_phase_extended_computations_include_includelist} is provided so only these extended_computation_fns will be used (overwritting the phase defaults!).')
	active_phase_dict['extended_computations_include_includelist'] = custom_phase_extended_computations_include_includelist

# active_phase_dict['extended_computations_include_includelist'].remove('wcorr_shuffle_analysis')
# active_phase_dict['should_freeze_pipeline_updates'] = True

## Build Slurm Scripts:
session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
batch_scripts_collection: BatchScriptsCollection = generate_batch_single_session_scripts(global_data_root_parent_path, session_batch_basedirs=session_basedirs_dict, included_session_contexts=included_session_contexts,
                                                                                        output_directory=scripts_output_path, use_separate_run_directories=True,
                                                                                        # should_freeze_pipeline_updates=False, 
                                                                                        # create_slurm_scripts=True, should_create_vscode_workspace=True,
                                                                                        # extended_computations_include_includelist=extended_computations_include_includelist,
                                                                                        force_recompute_override_computations_includelist=force_recompute_override_computations_includelist, # ['split_to_directional_laps', 'rank_order_shuffle_analysis'],
                                                                                        # should_perform_figure_generation_to_file=False,
                                                                                        # batch_session_completion_handler_kwargs=dict(enable_hdf5_output=False),
                                                                                        # custom_user_completion_functions=custom_user_completion_functions,
                                                                                        # custom_user_completion_function_template_code=custom_user_completion_function_template_code,
                                                                                        # should_force_reload_all=True,
                                                                                        # should_force_reload_all=False,
                                                                                        **active_phase_dict
                                                                                    )


# batch_scripts_collection.included_session_contexts, output_python_scripts, output_slurm_scripts

output_python_scripts = batch_scripts_collection.output_python_scripts
output_slurm_scripts = batch_scripts_collection.output_slurm_scripts
vscode_workspace_path = batch_scripts_collection.vscode_workspace_path
if vscode_workspace_path is not None:
    display(fullwidth_path_widget(vscode_workspace_path, file_name_label='vscode_workspace_path:'))

computation_script_paths = [x[0] for x in output_python_scripts]
generate_figures_script_paths = [x[1] for x in output_python_scripts]
print(f'generate_figures_script_paths: {generate_figures_script_paths}')
_out_widget = display_generated_scripts_ipywidget(batch_scripts_collection.included_session_contexts, output_python_scripts)
display(_out_widget)

generate_figures_script_paths: ['/home/halechr/cloud/turbo/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15/figures_kdiba_gor01_one_2006-6-08_14-26-15.py', '/home/halechr/cloud/turbo/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/figures_kdiba_gor01_one_2006-6-09_1-22-43.py', '/home/halechr/cloud/turbo/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-12_15-55-31/figures_kdiba_gor01_one_2006-6-12_15-55-31.py', '/home/halechr/cloud/turbo/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19/figures_kdiba_gor01_two_2006-6-07_16-40-19.py', '/home/halechr/cloud/turbo/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-08_21-16-25/figures_kdiba_gor01_two_2006-6-08_21-16-25.py', '/home/halechr/cloud/turbo/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-09_22-24-40/figures_kdiba_gor01_two_2006-6-09_22-24-40.py', '/home/halechr/cloud/turbo/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46/figures_kdiba_gor01_two_2006-6-12_16-53-46.py', '/ho

In [4]:
print(active_phase_dict['extended_computations_include_includelist'])

['lap_direction_determination', 'pf_computation', 'pfdt_computation', 'firing_rate_trends', 'extended_stats', 'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_inst_spike_rate_groups', 'long_short_endcap_analysis', 'split_to_directional_laps', 'merged_directional_placefields', 'rank_order_shuffle_analysis', 'directional_decoders_decode_continuous', 'directional_decoders_evaluate_epochs', 'directional_decoders_epoch_heuristic_scoring']


In [6]:
# Maximum number of parallel script executions
max_parallel_executions = 2
# List of your script paths
# if active_phase.value == ProcessingScriptPhases.figure_run:
if active_phase.is_figure_phase:
    print(f'fig mode!')
    script_paths = generate_figures_script_paths
else:
	script_paths = computation_script_paths

if (platform.system() == 'Windows'):
    powershell_script_path = build_windows_powershell_run_script(script_paths, max_concurrent_jobs=max_parallel_executions, script_name='run_scripts')
    # print(f'powershell_script_path: {powershell_script_path}')
    display(fullwidth_path_widget(powershell_script_path, file_name_label='powershell_script_path:'))


    if active_phase_dict['should_perform_figure_generation_to_file']:
        powershell_figures_script_path = build_windows_powershell_run_script(generate_figures_script_paths, max_concurrent_jobs=1, script_name='run_figure_scripts')
        display(fullwidth_path_widget(powershell_figures_script_path, file_name_label='powershell_figures_script_path:'))


if (platform.system() != 'Windows'):
    ## Linux Only: Slurm Scripts
    # sbatch_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts])
    sbatch_run_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts['run']])
    sbatch_figs_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts['figs']])

    if active_phase.is_figure_phase:
        # print(sbatch_figs_start_slurm_scripts)
        initial_code = sbatch_figs_start_slurm_scripts
    else:
        # print(sbatch_run_start_slurm_scripts)
        initial_code = sbatch_run_start_slurm_scripts

    # Create and display the code block widget
    slurm_code_block = code_block_widget(initial_code, label="Python Code:")



# Batch 1
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-02_17-46-44/run_kdiba_pin01_one_11-02_17-46-44.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-02_19-28-0/run_kdiba_pin01_one_11-02_19-28-0.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-03_12-3-25/run_kdiba_pin01_one_11-03_12-3-25.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_fet11-01_12-58-54/run_kdiba_pin01_one_fet11-01_12-58-54.sh'

# Batch 2:
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30/run_kdiba_vvp01_one_2006-4-09_17-29-30.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50/run_kdiba_vvp01_one_2006-4-10_12-25-50.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54/run_kdiba_vvp01_two_2006-4-09_16-40-54.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-10_12-58-3/run_kdiba_vvp01_two_2006-4-10_12-58-3.sh'

# Batch 3
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15/run_kdiba_gor01_one_2006-6-08_14-26-15.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/run_kdiba_gor01_one_2006-6-09_1-22-43.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19/run_kdiba_gor01_two_2006-6-07_16-40-19.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-08_21-16-25/run_kdiba_gor01_two_2006-6-08_21-16-25.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-09_22-24-40/run_kdiba_gor01_two_2006-6-09_22-24-40.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46/run_kdiba_gor01_two_2006-6-12_16-53-46.sh'


## Execute the generated scripts in parallel:

In [7]:
import subprocess
from concurrent.futures import ProcessPoolExecutor, as_completed
import ipywidgets as widgets
from IPython.display import display

## recieves: max_parallel_executions, script_paths
"""
# Maximum number of parallel script executions
max_parallel_executions = 5
# List of your script paths
script_paths = output_python_scripts
"""

# Function to execute a script
def run_script(script_path):
    python_executable = 'python'
    # python_executable = '/home/halechr/Library/VSCode/green/.venv_green/bin/python'
    
    try:
        result = subprocess.run([python_executable, script_path], capture_output=True, text=True)
        return script_path, result.stdout, result.stderr
    except Exception as e:
        return script_path, None, str(e)

# Create a progress bar
progress_bar = widgets.IntProgress(value=0, min=0, max=len(script_paths), description='Running:', bar_style='info')
display(progress_bar)

# Run scripts in parallel with a limit on the number of parallel instances
with ProcessPoolExecutor(max_workers=max_parallel_executions) as executor:
    futures = {executor.submit(run_script, path): path for path in script_paths}
    for future in as_completed(futures):
        script, stdout, stderr = future.result()
        progress_bar.value += 1  # Update the progress bar
        if stderr:
            print(f"Error in {script}: {stderr}")
        else:
            print(f"Completed {script}")

# Progress bar will automatically update as scripts complete



IntProgress(value=0, bar_style='info', description='Running:', max=15)

Error in /home/halechr/cloud/turbo/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/run_kdiba_gor01_one_2006-6-09_1-22-43.py: [neptune] [warning] /home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Batch/NeptuneAiHelpers.py:78: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'NoneType'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type

Error in /home/halechr/cloud/turbo/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15/run_kdiba_gor01_one_2006-6-08_14-26-15.py: [neptune] [warning] /home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Batch/NeptuneAiHelpers.py:78: NeptuneUnsupportedType: You're attempting to log a type that is not directly supporte

## Resume normal stuff:

In [8]:
active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default
# Hardcoded included_session_contexts:
included_session_contexts = UserAnnotationsManager.get_hardcoded_good_sessions()
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

# Output Paths:
included_h5_paths = [get_file_str_if_file_exists(v.pipeline_results_h5) for v in good_session_concrete_folders]
# copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, backup_mode=BackupMethods.RenameInSourceDirectory, only_include_file_types=['local_pkl', 'global_pkl'])
check_output_h5_files(included_h5_paths)
# from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import generate_batch_single_session_scripts

# ## Build Slurm Scripts:
# session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
# included_session_contexts, output_python_scripts, output_slurm_scripts = generate_batch_single_session_scripts(global_data_root_parent_path, session_batch_basedirs=session_basedirs_dict, included_session_contexts=included_session_contexts, output_directory=Path('output/generated_slurm_scripts/').resolve(), use_separate_run_directories=True, should_perform_figure_generation_to_file=False)
# display(output_python_scripts)

WARN: file_size < 0.01 for /home/halechr/cloud/turbo/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/pipeline_results.h5!
WARN: file_size < 0.01 for /home/halechr/cloud/turbo/Data/KDIBA/pin01/one/11-02_17-46-44/output/pipeline_results.h5!


,path,modification_time,creation_time,file_size
0,/home/halechr/cloud/turbo/Data/KDIBA/gor01/one...,2024-06-11 06:29:41,2024-06-11 06:29:41,1.07
1,/home/halechr/cloud/turbo/Data/KDIBA/gor01/one...,2024-06-11 06:30:50,2024-06-11 06:30:50,0.60
2,/home/halechr/cloud/turbo/Data/KDIBA/gor01/one...,2024-06-11 06:31:15,2024-06-11 06:31:15,0.24
3,/home/halechr/cloud/turbo/Data/KDIBA/gor01/two...,2024-06-11 06:35:52,2024-06-11 06:35:52,0.91
4,/home/halechr/cloud/turbo/Data/KDIBA/gor01/two...,2024-06-11 06:34:07,2024-06-11 06:34:07,0.44
5,/home/halechr/cloud/turbo/Data/KDIBA/gor01/two...,2024-06-11 06:38:27,2024-06-11 06:38:27,0.81
6,/home/halechr/cloud/turbo/Data/KDIBA/vvp01/one...,2024-06-11 06:38:13,2024-06-11 06:38:13,0.21
7,/home/halechr/cloud/turbo/Data/KDIBA/vvp01/one...,2024-06-11 06:39:48,2024-06-11 06:39:48,0.19
8,/home/halechr/cloud/turbo/Data/KDIBA/vvp01/two...,2024-06-11 06:39:58,2024-06-11 06:39:58,0.23
9,/home/halechr/cloud/turbo/Data/KDIBA/vvp01/two...,2024-06-11 06:41:14,2024-06-11 06:41:14,0.17


In [9]:
from pyphocorehelpers.Filesystem.path_helpers import save_filelist_to_text_file

## INPUTS: good_session_concrete_folders, target_dir, BATCH_DATE_TO_USE

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path

included_h5_paths = [Path(get_file_str_if_file_exists(v.pipeline_results_h5)).resolve() for v in good_session_concrete_folders]
check_output_h5_files(included_h5_paths)
included_h5_paths

def _across_session_h5_output_basename_fn(session_context: Optional[IdentifyingContext], session_descr: Optional[str], basename: str, *args, separator_char: str = "_"):
    """ Captures `BATCH_DATE_TO_USE` """
    # a_session_folder.context
    if session_context is not None:
        session_descr = session_context.session_name # '2006-6-07_16-40-19'
    _filename_list = [BATCH_DATE_TO_USE, session_descr, basename]
    if len(args) > 0:
        _filename_list.extend([str(a_part) for a_part in args if a_part is not None])
    return separator_char.join(_filename_list)

copy_h5_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, target_dir=collected_outputs_path, backup_mode=BackupMethods.CommonTargetDirectory, rename_backup_basename_fn=_across_session_h5_output_basename_fn, only_include_file_types=['h5']) # , rename_backup_suffix=BATCH_DATE_TO_USE
copy_h5_dict


## INPUTS: target_dir, BATCH_DATE_TO_USE
h5_filelist_output_filename=f'{BATCH_DATE_TO_USE}_all_sessions_h5_filelist.txt'
h5_filelist_output_file_path = Path(target_dir).joinpath(h5_filelist_output_filename).resolve() # Use Default
print(f'h5_filelist_output_file_path: {h5_filelist_output_file_path}')
_out_string, filelist_path = save_filelist_to_text_file(included_h5_paths, filelist_path=h5_filelist_output_file_path, debug_print=True) # r"W:\Data\all_sessions_h5_filelist_2024-03-28_Apogee.txt"
filelist_path

WARN: file_size < 0.01 for /home/halechr/cloud/turbo/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/pipeline_results.h5!
WARN: file_size < 0.01 for /home/halechr/cloud/turbo/Data/KDIBA/pin01/one/11-02_17-46-44/output/pipeline_results.h5!
h5_filelist_output_file_path: /home/halechr/cloud/turbo/Data/Output/collected_outputs/2024-06-11_Lab_all_sessions_h5_filelist.txt
/home/halechr/cloud/turbo/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5
/home/halechr/cloud/turbo/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5
/home/halechr/cloud/turbo/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5
/home/halechr/cloud/turbo/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/pipeline_results.h5
/home/halechr/cloud/turbo/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/pipeline_results.h5
/home/halechr/cloud/turbo/Data/KDIBA/gor01/two/2006-6-09_22-24-40/output/pipeline_results.h5
/home/halechr/cloud/turbo/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/pi

PosixPath('/home/halechr/cloud/turbo/Data/Output/collected_outputs/2024-06-11_Lab_all_sessions_h5_filelist.txt')

In [ ]:
copy_h5_dict

In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import read_filelist_from_text_file
    
an_h5_filelist_path = Path(r'L:/Scratch/collected_outputs/2024-03-28_Apogee_all_sessions_h5_filelist.txt').resolve()
read_hdf5_output_paths = read_filelist_from_text_file(filelist_path=an_h5_filelist_path, debug_print=False)
read_hdf5_output_paths


# Output File Processing Helpers

In [ ]:
## INPUT a_copy_dict
a_copy_dict = copy_h5_dict

moved_files_dict_h5_files = copy_movedict(a_copy_dict)
moved_files_dict_h5_files
# INPUTS: active_filelist_prefix, target_dir
# active_filelist_prefix: str = 'backed_up_files'
active_filelist_prefix: str = 'session_h5_files'

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path

moved_files_copydict_output_filename=f'{active_filelist_prefix}_copydict_{BATCH_DATE_TO_USE}.csv'
moved_files_copydict_file_path = target_dir.joinpath(moved_files_copydict_output_filename).resolve() # Use Default
print(f'moved_files_copydict_file_path: "{moved_files_copydict_file_path}"')

_out_string, filedict_out_path = save_copydict_to_text_file(moved_files_dict_h5_files, moved_files_copydict_file_path, debug_print=True)

In [ ]:
## INPUTS: moved_files_copydict_file_path
moved_files_copydict_file_path = Path(r"W:\Data\session_h5_files_copydict_2024-03-28_Apogee.csv").resolve()
assert moved_files_copydict_file_path.exists()

read_moved_files_dict_files = read_copydict_from_text_file(moved_files_copydict_file_path, debug_print=False)
read_moved_files_dict_files

In [ ]:
# read_moved_files_dict_files
restore_moved_files_dict_files = invert_filedict(read_moved_files_dict_files)
restore_moved_files_dict_files

## Extract `across_sessions_instantaneous_fr_dict` from the computation outputs

In [11]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True, )
(neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths, override_output_parent_path=global_data_root_parent_path, output_path_suffix=f'{BATCH_DATE_TO_USE}')

failed for file path: /home/halechr/cloud/turbo/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/pipeline_results.h5, table_key: /kdiba/gor01/two/2006-6-12_16-53-46/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/two/2006-6-12_16-53-46/neuron_identities/table``. Skipping.
failed for file path: /home/halechr/cloud/turbo/Data/KDIBA/pin01/one/11-02_17-46-44/output/pipeline_results.h5, table_key: /kdiba/pin01/one/11-02_17-46-44/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/pin01/one/11-02_17-46-44/neuron_identities/table``. Skipping.
concatenating dataframes from 13 of 15 files
failed for file path: /home/halechr/cloud/turbo/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/pipeline_results.h5, table_key: /kdiba/gor01/two/2006-6-12_16-53-46/global_computations/long_short_fr_indicies_analysis/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/two/2006-6-12_16-53-46/global_computations

In [12]:
# Options
session_identifier_key: str = 'session_name'
# session_identifier_key: str = 'session_datetime'

## !IMPORTANT! Count of the fields of interest using .value_counts(...) and converting to an explicit pd.DataFrame:
# _out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
# _out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'count']
_out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership','is_refined_LxC', 'is_refined_SxC'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
_out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'is_refined_LxC', 'is_refined_SxC', 'count']
_out_value_counts_df

,format_name,animal,session_name,session_datetime,track_membership,is_refined_LxC,is_refined_SxC,count
0,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,LEFT_ONLY,True,False,1
1,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,RIGHT_ONLY,False,True,5
2,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,SHARED,False,False,58
3,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,LEFT_ONLY,False,False,1
4,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,LEFT_ONLY,True,False,6
5,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,RIGHT_ONLY,False,False,1
6,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,RIGHT_ONLY,False,True,6
7,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,SHARED,False,False,94
8,kdiba,gor01,2006-6-08_21-16-25,2006-06-08 21:16:25,LEFT_ONLY,False,False,10
9,kdiba,gor01,2006-6-08_21-16-25,2006-06-08 21:16:25,RIGHT_ONLY,False,False,3


In [13]:
## Find the time of the first session for each animal:
first_session_time  = _out_value_counts_df.groupby(['animal']).agg(session_datetime_first=('session_datetime', 'first')).reset_index()

## Subtract this initial time from all of the 'session_datetime' entries for each animal:
# Merge the first session time back into the original DataFrame
merged_df = pd.merge(_out_value_counts_df, first_session_time, on='animal')

# Subtract this initial time from all of the 'session_datetime' entries for each animal
merged_df['time_since_first_session'] = merged_df['session_datetime'] - merged_df['session_datetime_first']

merged_df

,format_name,animal,session_name,session_datetime,track_membership,is_refined_LxC,is_refined_SxC,count,session_datetime_first,time_since_first_session
0,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,LEFT_ONLY,True,False,1,2006-06-07 16:40:19,0 days 00:00:00
1,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,RIGHT_ONLY,False,True,5,2006-06-07 16:40:19,0 days 00:00:00
2,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,SHARED,False,False,58,2006-06-07 16:40:19,0 days 00:00:00
3,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,LEFT_ONLY,False,False,1,2006-06-07 16:40:19,0 days 21:45:56
4,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,LEFT_ONLY,True,False,6,2006-06-07 16:40:19,0 days 21:45:56
5,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,RIGHT_ONLY,False,False,1,2006-06-07 16:40:19,0 days 21:45:56
6,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,RIGHT_ONLY,False,True,6,2006-06-07 16:40:19,0 days 21:45:56
7,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,SHARED,False,False,94,2006-06-07 16:40:19,0 days 21:45:56
8,kdiba,gor01,2006-6-08_21-16-25,2006-06-08 21:16:25,LEFT_ONLY,False,False,10,2006-06-07 16:40:19,1 days 04:36:06
9,kdiba,gor01,2006-6-08_21-16-25,2006-06-08 21:16:25,RIGHT_ONLY,False,False,3,2006-06-07 16:40:19,1 days 04:36:06


In [14]:
import matplotlib.pyplot as plt

point_size = 8
df = _out_value_counts_df.copy()
animals = df['animal'].unique()
track_memberships = df['track_membership'].unique()

fig, axes = plt.subplots(1, len(animals), figsize=(15, 5))

for i, animal in enumerate(animals):
	ax = axes[i]
	subset_df = df[df['animal'] == animal]
	
	for track_membership in track_memberships:
		track_subset_df = subset_df[subset_df['track_membership'] == track_membership]
		ax.plot(track_subset_df['session_datetime'], track_subset_df['count'], label=f'Track: {track_membership}')
		ax.scatter(track_subset_df['session_datetime'], track_subset_df['count'], s=point_size)
		
	ax.set_title(f'Animal: {animal}')
	ax.set_xlabel('Session Datetime')
	ax.set_ylabel('Count')
	ax.legend()

plt.tight_layout()
plt.show()

/tmp/ipykernel_1047084/4006058737.py:25: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [15]:
_out_value_counts_df

,format_name,animal,session_name,session_datetime,track_membership,is_refined_LxC,is_refined_SxC,count
0,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,LEFT_ONLY,True,False,1
1,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,RIGHT_ONLY,False,True,5
2,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,SHARED,False,False,58
3,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,LEFT_ONLY,False,False,1
4,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,LEFT_ONLY,True,False,6
5,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,RIGHT_ONLY,False,False,1
6,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,RIGHT_ONLY,False,True,6
7,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,SHARED,False,False,94
8,kdiba,gor01,2006-6-08_21-16-25,2006-06-08 21:16:25,LEFT_ONLY,False,False,10
9,kdiba,gor01,2006-6-08_21-16-25,2006-06-08 21:16:25,RIGHT_ONLY,False,False,3


In [16]:


## See if the number of cells decreases over re-exposures to the track
df = _out_value_counts_df[_out_value_counts_df['animal'] == 'gor01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'pin01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'vvp01']

# Sort by column: 'session_datetime' (ascending)
df = df.sort_values(['session_datetime'])

'LEFT_ONLY'

# df.to_clipboard(index=False)
df

,format_name,animal,session_name,session_datetime,track_membership,is_refined_LxC,is_refined_SxC,count
0,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,LEFT_ONLY,True,False,1
1,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,RIGHT_ONLY,False,True,5
2,kdiba,gor01,2006-6-07_16-40-19,2006-06-07 16:40:19,SHARED,False,False,58
3,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,LEFT_ONLY,False,False,1
4,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,LEFT_ONLY,True,False,6
5,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,RIGHT_ONLY,False,False,1
6,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,RIGHT_ONLY,False,True,6
7,kdiba,gor01,2006-6-08_14-26-15,2006-06-08 14:26:15,SHARED,False,False,94
8,kdiba,gor01,2006-6-08_21-16-25,2006-06-08 21:16:25,LEFT_ONLY,False,False,10
9,kdiba,gor01,2006-6-08_21-16-25,2006-06-08 21:16:25,RIGHT_ONLY,False,False,3


In [17]:
## Get the number of cells in each session of the animal:
num_LxCs = df[df['track_membership'] == 'LEFT_ONLY']['count'].to_numpy()
num_Shared = df[df['track_membership'] == 'SHARED']['count'].to_numpy()
num_SxCs = df[df['track_membership'] == 'RIGHT_ONLY']['count'].to_numpy()

num_TotalCs = num_LxCs + num_Shared + num_SxCs
num_TotalCs

ValueError: operands could not be broadcast together with shapes (8,) (6,) 

In [ ]:
# The only safe point to align each session to is the switchpoint (the delta):


In [ ]:
# Each session can be expressed in terms of time from the start of the first session.


In [ ]:
df.plot()


In [18]:

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)

NameError: name 'num_sessions' is not defined

## Extract output files from all completed sessions:

# 2023-10-06 - `joined_neruon_fri_df` loading

In [ ]:
# BATCH_DATE_TO_USE = '2023-10-05_NewParameters'
BATCH_DATE_TO_USE = '2023-10-07'
all_sessions_joined_neruon_fri_df, out_path = build_and_merge_all_sessions_joined_neruon_fri_df(global_data_root_parent_path, BATCH_DATE_TO_USE)


In [ ]:

joined_neruon_fri_df_basename = f'{BATCH_DATE_TO_USE}_{output_file_prefix}_joined_neruon_fri_df'
AcrossSessionTables.write_table_to_files(joined_neruon_fri_df, global_data_root_parent_path=global_data_root_parent_path, output_basename=joined_neruon_fri_df_basename, include_csv=False)
print(f'>>\t done with {output_file_prefix}')

# 2023-10-04 - Load Saved across-sessions-data, process, and produce figures

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=global_data_root_parent_path, BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, plotting_enabled=True)


 #TODO 2023-10-05 11:40: - [ ] Extract the "contrarian cells", the ones that have a strong exclusivity on the laps but the opposite tendency on the replays


## 2023-11-15 - For manual testing

In [ ]:
# ## Load the saved across-session results:
# Outputs: across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list, neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table

inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')
# neuron_replay_stats_table


# 2024-01-18 - Extracts the callback results 'determine_computation_datetimes_completion_function':


In [ ]:
extracted_callback_fn_results = {a_sess_ctxt:a_result.across_session_results.get('determine_session_t_delta_completion_function', {}) for a_sess_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None}
extracted_callback_fn_results